<a href="https://colab.research.google.com/github/ChanceChallacombe/Airquality-as-Affected-by-Coronavirus-Shutdown/blob/master/esm_workbork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! tensorflow --version

/bin/bash: tensorflow: command not found


In [2]:
! pip install torch torchvision tensorflow accelerate transformers datasets huggingface_hub evaluate datasets requests pandas sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#model_checkpoint = "facebook/esm2_t12_35M_UR50D"

In [4]:
import requests
url2="https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession%2Cprotein_name%2Ccc_function%2Corganism_name%2Csequence%2Clength&format=tsv&query=%28%28keyword%3AKW-0800%29%29%20AND%20%28length%3A%5B1%20TO%20200%5D%29%20AND%20%28existence%3A1%29"

In [5]:
uniprot_request2 = requests.get(url2)

In [6]:
from io import BytesIO
import pandas

In [7]:
bio2 = BytesIO(uniprot_request2.content)

df2 = pandas.read_csv(bio2, compression='gzip', sep='\t')
df2

,Entry,Protein names,Function [CC],Organism,Sequence,Length
0,A0A088MIT0,Bradykinin-related peptides [Cleaved into: Nat...,"FUNCTION: [[Val1,Thr6]-bradykinyl-Ser,Pro,Ala]...",Physalaemus nattereri (Cuyaba dwarf frog) (Eup...,MAFLKKSLFLVLFLGVVSLSFCEEEKREEHEEEKRDEEDAESLGKR...,134
1,A0A0B5A8P4,Con-Ins G3 (Insulin 3) [Cleaved into: Con-Ins ...,"FUNCTION: This venom insulin, from a fish-hunt...",Conus geographus (Geography cone) (Nubecula ge...,MTTSFYFLLVALGLLLYVCQSSFGNQHTRNSDTPKHRCGSELADQY...,113
2,A0A0B5AC95,Con-Ins G1a (Insulin 1) [Cleaved into: Con-Ins...,"FUNCTION: This venom insulin, from a fish-hunt...",Conus geographus (Geography cone) (Nubecula ge...,MTTSSYFLLMALGLLLYVCQSSFGNQHTRTFDTPKHRCGSEITNSY...,115
3,A0A0N7CSQ4,Tau-scoloptoxin(04)-Sm1b (Tau-SLPTX(04)-Sm1b) ...,FUNCTION: [Tau-scoloptoxin(04)-Sm1a]: Extremel...,Scolopendra mutilans (Chinese red-headed centi...,MLKSFCILSVFMVLFLAKFPDLCSGEEISPLKIVVRNSEYLNNPCN...,68
4,A0A2L0ART2,Mu-scoloptoxin(15)-Ssm1a (Mu-SLPTX(15)-Ssm1a) ...,FUNCTION: Blocks voltage-gated potassium chann...,Scolopendra mutilans (Chinese red-headed centi...,MEKKIIFLVFLVALLALPGFISTEVIKKDTPYKKRKFPYKSECLKA...,76
...,...,...,...,...,...,...
3708,Q9R578,01-ST-4=HEAT-stable enterotoxin,NaN,Vibrio cholerae,FIKQVDENGNLIDCCEICCNPACFGCLN,28
3709,Q9R579,01-ST-3=HEAT-stable enterotoxin,NaN,Vibrio cholerae,NLIDCCEICCNPACFGCLN,19
3710,Q9R580,"01-ST-2, VC-H-ST=HEAT-stable enterotoxin",NaN,Vibrio cholerae,LIDCCEICCNPACFGCLN,18
3711,Q9R581,"01-ST-1, NAG-ST, VM-ST=HEAT-stable enterotoxin",NaN,Vibrio cholerae,IDCCEICCNPACFGCLN,17


In [8]:

import gc, torch
from accelerate import Accelerator
gc.collect()
torch.cuda.empty_cache()

In [9]:
from transformers import AutoTokenizer, EsmForProteinFolding, EsmModel

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")

model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1", low_cpu_mem_usage=True)
accelerator = Accelerator()
device = accelerator.device
model = model.to(device)

In [10]:

model.esm = model.esm.half()
torch.backends.cuda.matmul.allow_tf32 = True
model.trunk.set_chunk_size(64)

In [11]:
testseq=df2["Sequence"][0]

In [13]:
test_input=tokenizer([df2["Sequence"][0]], return_tensors="pt", add_special_tokens=False)['input_ids']
test_input = test_input.cuda()

In [14]:
import torch

with torch.no_grad():
    output = model(test_input)

In [15]:
output

EsmForProteinFoldingOutput(frames=tensor([[[[ 7.9013e-01, -3.1440e-01, -3.8051e-01,  ..., -2.9701e+00,
           -3.0997e+01, -5.1061e+01],
          [ 4.9123e-01,  2.2177e-01, -1.6800e-01,  ..., -8.6439e-01,
           -2.8930e+01, -5.0668e+01],
          [ 2.5433e-01, -6.6565e-01, -2.1913e-01,  ..., -2.7908e+00,
           -2.6474e+01, -4.8974e+01],
          ...,
          [ 2.0786e-01, -8.9769e-01,  1.6368e-01,  ...,  3.7429e-01,
            1.4179e+01,  2.6077e+01],
          [ 6.5286e-01, -2.2147e-01, -5.1353e-01,  ...,  2.1724e+00,
            1.4425e+01,  2.5849e+01],
          [ 4.3864e-01, -8.7579e-01,  4.5751e-03,  ...,  2.0239e+00,
            1.5560e+01,  2.7525e+01]]],


        [[[ 7.9364e-01, -3.1600e-01, -3.9144e-01,  ..., -2.0240e+00,
           -3.0509e+01, -4.9106e+01],
          [ 4.9436e-01,  2.5596e-01, -1.5472e-01,  ...,  7.5568e-02,
           -2.7873e+01, -4.8814e+01],
          [ 2.3241e-01, -6.5219e-01, -1.9963e-01,  ..., -2.3462e+00,
           -2.5782e+01